In [234]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import sounddevice as sd
import ipywidgets as widgets

# reading in data
df = pd.read_csv('CovidFaelle_Timeline_Var2.csv', sep=';')

# displaying first ten, checking columns
df.head(10)
lof = list(df.columns)


In [235]:
# cleaning data

# changing to proper floats
df['SiebenTageInzidenzFaelle'] = df['SiebenTageInzidenzFaelle'].str.replace(',', '.')
df['SiebenTageInzidenzFaelle'].head(20)

# getting rid of every tenth value (whole of austria)
df = df.loc[df['BundeslandID'] % 10 != 0, :].reset_index()
# df.tail(10)


In [236]:
# order of counties according to longitude values
# rearrange dataframe

longitude_df = pd.read_html('https://www.distancelatlong.com/country/austria')

longitude_df[2].loc[:, ['States', 'Longitude']]

longitude2 = [longitude_df[2].loc[i, ['States']] for i in range(9)]

longitude3 = [(y, x) for x, y in enumerate(longitude2)]

longitude4 = [(longitude3[i][0][0].split()[0], longitude3[i][1]) for i in range(len(longitude3))]

long_dict = {longitude4[j]:
            longitude_df[2].loc[j, 'Longitude']
            for j in range(len(longitude_df[2]))}

long_dict_sort = sorted(long_dict.items(), key=lambda x: x[1])

long_dict_sort_keys = [c[0] for c in long_dict_sort]

long_sort_index = [d[1] for d in long_dict_sort_keys]

long_sort_index_l = long_sort_index.copy()

l_s_l = np.array(long_sort_index)

l_s_l_2 = []

In [237]:
# creating new shape for data frame in the same length as data frame

for i in range(len(df) // 9):
    l_s_l_temp = l_s_l + i * 9
    l_s_l_2.append(l_s_l_temp)
    
    
l_s_l_3 = np.concatenate(l_s_l_2)

In [238]:
df2 = df.copy()
df3 = df2.reindex(l_s_l_3)
df = df3

In [239]:
df.tail(9)

,index,Time,Bundesland,BundeslandID,AnzEinwohner,AnzahlFaelle,AnzahlFaelleSum,AnzahlFaelle7Tage,SiebenTageInzidenzFaelle,AnzahlTotTaeglich,AnzahlTotSum,AnzahlGeheiltTaeglich,AnzahlGeheiltSum
5011,5567,04.09.2021 00:00:00,Vorarlberg,8,399237,63,32458,464,116.2217,0,310,28,31362
5010,5566,04.09.2021 00:00:00,Tirol,7,760105,61,66939,730,96.03936,0,685,92,65180
5008,5564,04.09.2021 00:00:00,Salzburg,5,560710,123,54023,906,161.5809,0,601,69,52019
5007,5563,04.09.2021 00:00:00,Oberösterreich,4,1495608,299,125904,2284,152.7138,2,1662,137,120123
5005,5561,04.09.2021 00:00:00,Kärnten,2,562089,62,42131,449,79.88058,0,828,41,40698
5009,5565,04.09.2021 00:00:00,Steiermark,6,1247077,194,85143,1055,84.59782,1,2105,86,81292
5006,5562,04.09.2021 00:00:00,Niederösterreich,3,1690879,212,114672,1610,95.21675,0,1697,181,110259
5012,5568,04.09.2021 00:00:00,Wien,9,1920949,501,152294,3391,176.5273,1,2367,236,139716
5004,5560,04.09.2021 00:00:00,Burgenland,1,296010,32,19003,261,88.1727,0,334,16,18255


In [240]:
# reseting index of df

df.reset_index()

,level_0,index,Time,Bundesland,BundeslandID,AnzEinwohner,AnzahlFaelle,AnzahlFaelleSum,AnzahlFaelle7Tage,SiebenTageInzidenzFaelle,AnzahlTotTaeglich,AnzahlTotSum,AnzahlGeheiltTaeglich,AnzahlGeheiltSum
0,7,7,26.02.2020 00:00:00,Vorarlberg,8,399237,0,0,0,0,0,0,0,0
1,6,6,26.02.2020 00:00:00,Tirol,7,760105,0,0,0,0,0,0,0,0
2,4,4,26.02.2020 00:00:00,Salzburg,5,560710,0,0,0,0,0,0,0,0
3,3,3,26.02.2020 00:00:00,Oberösterreich,4,1495608,0,0,0,0,0,0,0,0
4,1,1,26.02.2020 00:00:00,Kärnten,2,562089,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5008,5005,5561,04.09.2021 00:00:00,Kärnten,2,562089,62,42131,449,79.88058,0,828,41,40698
5009,5009,5565,04.09.2021 00:00:00,Steiermark,6,1247077,194,85143,1055,84.59782,1,2105,86,81292
5010,5006,5562,04.09.2021 00:00:00,Niederösterreich,3,1690879,212,114672,1610,95.21675,0,1697,181,110259
5011,5012,5568,04.09.2021 00:00:00,Wien,9,1920949,501,152294,3391,176.5273,1,2367,236,139716


In [241]:
# scaling values for volume, size and color

BIDS = list(df['BundeslandID'][:9])
AmpSize = [np.array(df.loc[(df['BundeslandID'] == bid), 'AnzahlFaelle'])
           for bid in BIDS]


Color = [np.array(df.loc[(df['BundeslandID'] == bid),
                         'SiebenTageInzidenzFaelle']) for bid in BIDS]


Pitch = [np.array(df.loc[(df['BundeslandID'] == bid),
                         'SiebenTageInzidenzFaelle']) for bid in BIDS]

In [242]:
# Converter function to floats


def strtofloat(a):
    for i in range(len(a)):
        a[i] = float(a[i])
    return a


Color2 = [np.apply_along_axis(strtofloat, 0, Color[j])
          for j in range(len(Color))]

# scale funtion for 'colorchanges' and pitchchanges
Pitch2 = [np.apply_along_axis(strtofloat, 0, Pitch[j])
          for j in range(len(Pitch))]
Pitch2


def pv(ar):
    for i in range(len(ar)):
        if ar[i] < 50:
            ar[i] = 1
        elif 50 < ar[i] < 100:
            ar[i] = 1.5
        elif 100 < ar[i] < 150:
            ar[i] = 2
        elif 150 < ar[i] < 250:
            ar[i] = 2.5
        elif ar[i] > 250:
            ar[i] = 3
    return ar


Pitches = [np.apply_along_axis(pv, 0, Pitch2[j])
           for j in range(len(Pitch2))]


def cc(ar):
    for i in range(len(ar)):
        if ar[i] < 50:
            ar[i] = 'lightgreen'
        elif 50 < ar[i] < 100:
            ar[i] = 'green'
        elif 100 < ar[i] < 150:
            ar[i] = 'yellow'
        elif 150 < ar[i] < 250:
            ar[i] = 'orange'
        elif ar[i] > 250:
            ar[i] = 'red'
    return ar


Color3 = [np.apply_along_axis(cc, 0, Color2[j])
          for j in range(len(Color2))]


In [243]:
# last preparations

AmpSize1 = []
for i in range(len(AmpSize[0])):
    for j in range(len(AmpSize)):
        AmpSize1.append(AmpSize[j][i])
AmpSize1

Color4 = []
for x in range(len(Color3[0])):
    for y in range(len(Color3)):
        Color4.append(Color3[y][x])


In [244]:
# Audio Engine

def puresine(freq, dur, phase):
    sr = 44100
    phase1 = phase * np.pi
    t = np.arange(dur * sr) / sr
    sine = 1 * np.sin(2 * np.pi * freq * t + phase1)
    return sine


# simple panning - algorithm
def panner(x, angle):
    # pan a mono audio source into stereo
    # x is a numpy array, angle is the angle in radiants
    left = np.sqrt(2)/2.0 * (np.cos(angle) - np.sin(angle)) * x
    right = np.sqrt(2)/2.0 * (np.cos(angle) + np.sin(angle)) * x
    return np.dstack((left, right))[0]

In [245]:
# check for available audio devices

sd.query_devices()

  0 BlackHole 16ch, Core Audio (16 in, 16 out)
> 1 MacBook Pro Mikrofon, Core Audio (1 in, 0 out)
< 2 MacBook Pro Lautsprecher, Core Audio (0 in, 2 out)
  3 QuickTime Player Input, Core Audio (16 in, 16 out)
  4 Screen Record w/Audio, Core Audio (0 in, 2 out)

In [246]:
# set preferred audio output device

sd.default.device = 'BlackHole 16ch, Core Audio'

In [247]:
# Scaling to values between 0 and 1
Amps = [np.array(df.loc[(df['BundeslandID'] == bid), 'AnzahlFaelle'])
        for bid in BIDS]
Amps2 = [Amps[i] / Amps[i].max() for i in range(len(Amps))]


# setting samplerate, duration and time between datapoints

sr = 44100
splits = len(Amps2[0])
dur = splits / 5
global line
line = int(round((sr * dur) / splits, 0))
Amps2N = [np.append(Amps2[u], [0]) for u in range(len(Amps2))]
Pitches2 = [np.append(Pitches[u], [Pitches[u][-1]])
            for u in range(len(Pitches))]

In [248]:
# set base-frequency
slider = widgets.IntSlider(value=95, min=55, max=220, step=1, description='basefrequency')
display(slider)

IntSlider(value=95, description='basefrequency', max=220, min=55)

In [249]:
# choose mood of sonification

menu = widgets.RadioButtons(
        options=['light', 'dark', 'neutral'],
        value='neutral',
        description='Mood of Sonification',
        disabled=False
                    )


In [250]:
menu

RadioButtons(description='Mood of Sonification', index=2, options=('light', 'dark', 'neutral'), value='neutral…

In [251]:
latitude_df = pd.read_html('https://www.distancelatlong.com/country/austria')
latitude_df[2].loc[:, ['States', 'Latitude']]

bf = slider.value

if menu.value == 'neutral':
    basefreqs = [bf, bf * 1.5, bf * 2, bf * 4 * (3/4),
                 bf * 4 * (9/8), bf * 5, bf * 4 * (15/8), bf * 8, bf * 9]
elif menu.value == 'light':
    basefreqs =  [bf * 1, bf * 1.5, bf * 2.5, bf * 3, 
                  bf * 3.75, bf * 4.5,  bf * 5.625, bf * 6, bf * 7.5]
elif menu.value == 'dark':
    basefreqs =  [bf * 1, bf * 2, bf * 2.4, bf * 3, 
                  bf * 3.5, bf * 4.5, bf * 4.75, bf * 6, bf * 7]
    



lat_dict = {latitude_df[2].loc[i, 'States']:
            latitude_df[2].loc[i, 'Latitude']
            for i in range(len(latitude_df[2]))}


lat_dict_sort = sorted(lat_dict.items(), key=lambda x: x[1])


zipped_lat_freq = list(zip(lat_dict_sort, basefreqs))


basefreqs_lat = [sorted(zipped_lat_freq)[i][1]
                 for i in range(len(zipped_lat_freq))]


In [252]:
# make data frame, reorganise with reindex

d = pd.DataFrame({'states': lat_dict.keys(), 'freqs': basefreqs_lat})

d1 = d.reindex(l_s_l)

# newly ordered frequencies

d1

,states,freqs
7,Vorarlberg (1),285.0
6,Tirol (1),190.0
4,Salzburg (1),427.5
3,Oberosterreich (2),855.0
1,Karnten (1),95.0
5,Steiermark (1),142.5
2,Niederosterreich (1),475.0
8,Wien (1),760.0
0,Burgenland (1),712.5


In [253]:
basefreqs_lat1 = list(d1['freqs'])

Pitches3 = [Pitches2[i] * basefreqs_lat1[i] for i in range(len(Pitches2))]

pitch = [np.concatenate([np.linspace(Pitches3[j][i], Pitches3[j][i + 1], line)
                         for i in range(len(Pitches3[j]) - 1)])
         for j in range(len(Pitches3))]


env = [np.concatenate([np.linspace(Amps2N[j][i], Amps2N[j][i + 1], line)
                       for i in range(len(Amps2N[j]) - 1)])
       for j in range(len(Amps2N))]

In [254]:
def summation(callback, freqs):
    # Cumulative Sum
    phaseY = np.cumsum(freqs)
    # sin (cumulative sum (f) )
    x = np.sin((phaseY) * np.pi * 2 / sr)
    return x


def log_env(center_t, dy, speed, x):
    # type logistic
    return dy/(1+np.exp(-speed * (x - center_t)))


longsines = [summation(np.sin, log_env(0.25, pitch[i], 12, line))
             * env[i] for i in range(len(pitch))]


In [255]:
# reorganizing values for y-axis of scatter-plot 

d3 = pd.DataFrame({'states': lat_dict.keys(), 'lat': list(latitude_df[2].loc[:, 'Latitude'])})

d4 = d3.reindex(l_s_l)

d4.states = [i.split()[0] if i.split()[0] != 'Karnten' else 'Kärnten' for i in d1['states']]

In [256]:
d5 = {d4.loc[i, 'states']: d4.loc[i, 'lat'] for i in d4.index}

# sort - assign 1 - 9, - rearrange

d5_sort = sorted(d5.items(), key=lambda x: x[1])

d5_sort_2 = {x: y for y, x in enumerate(d5_sort)}

In [257]:
df_sort = pd.DataFrame({'c': d5_sort_2.keys(), 'a': d5_sort_2.values()})

In [258]:
df_sort_5 = {}

for c in df['Bundesland'][:9]:
    for d in range(len(df_sort['a'])):
        if c[:4] == df_sort['c'][d][0][:4]:
            df_sort_5[c] = df_sort['a'][d]


In [259]:
df_sort_5

{'Vorarlberg': 3,
 'Tirol': 2,
 'Salzburg': 4,
 'Oberösterreich': 8,
 'Kärnten': 0,
 'Steiermark': 1,
 'Niederösterreich': 5,
 'Wien': 7,
 'Burgenland': 6}

In [260]:
# plot

plt.style.available
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(30, 14))
ax.set_xticks(list(range(1, 10)))
ax.set_xticklabels(list(df['Bundesland'][:9]), fontsize=23)
ax.set_ylim(-2, 12)
ax.set_frame_on(False)
ax.axes.get_yaxis().set_visible(True)
ax.axes.get_xaxis().set_visible(True)
ax.set_yticklabels([])
ax.grid(False, axis='both')
ax.set_title('Covid19_Cases_in_Austria', fontsize=25)

x = np.array(list(range(1, 10)))

scaled_lat = list(zip(lat_dict_sort, list(range(1, 10))))
scaled_lat2 = sorted(scaled_lat)
scaled_lat3 = [i[1] for i in scaled_lat2]

y = np.array(list(df_sort_5.values()))


lines = ax.scatter(x, y,
                   marker='o',
                   s=50,
                   c='green', alpha=0.8)

plt.close()

In [261]:
def animate(i):
    lines.set_sizes(np.array(AmpSize1[i:i+9]) * 5)
    lines.set_color(Color4[i:i+9])
    ax.set_ylabel(df['Time'][i][:10], fontsize=23)
    if i == len(df) - 9:
        sd.play((panner(longsines[0], np.radians(-40)) +
                 panner(longsines[1], np.radians(-30)) +
                 panner(longsines[2], np.radians(-20)) +
                 panner(longsines[3], np.radians(-10)) +
                 panner(longsines[4], np.radians(0)) +
                 panner(longsines[5], np.radians(10)) +
                 panner(longsines[6], np.radians(20)) +
                 panner(longsines[7], np.radians(30)) +
                 panner(longsines[8], np.radians(40))) * 0.11, sr)
    return lines,


animation = FuncAnimation(fig, func=animate,
                          frames=np.arange(27, len(df), 9),
                          interval=(dur / splits) * 1000,
                          blit=False, repeat=False)


HTML(animation.to_html5_video())

In [262]:
# sd.stop()